# Psych 198: Reproducibility DeCal (Spring 2021)

## Demo/Lab 2: Cross Validation

In this demo/lab, we will go through the a typical process for cross validating a linear model. Code adopted from Psych 102.

Author: Yuyang Zhong (2020). This work is licensed under a [Creative Commons BY-NC-SA 4.0 International
License][cc-by]. 

![CC BY-NC-SA 4.0][cc-by-shield]

[cc-by]: http://creativecommons.org/licenses/by/4.0/
[cc-by-shield]: https://img.shields.io/badge/license-CC--BY--NC--SA%204.0-blue

#### Note on using Jupyter Notebooks 
Enter code into a code cell, then press SHIFT+Enter to run that cell. The output of the code should be shown right underneath the cell you just run.

In [ ]:
install.packages('psychTools')
install.packages('caTools')

library(car)
library(psychTools)
library(caTools)

### The Dataset

We will be using the The Motivational State Questionnaire (MSQ). If you are interested in looking more into this dataset, run the code `?msq`.

Let's take a look at this dataset.

In [ ]:
head(msq)

#### Removing duplicates

In [ ]:
sel <- complete.cases(msq[,'idle']) & complete.cases(msq[,'inspired']) & complete.cases(msq[, 'intense']) &
       complete.cases(msq[,'interested']) & complete.cases(msq[,'irritable']) & 
       complete.cases(msq[,'satisfied']) & complete.cases(msq[,'scared']) & complete.cases(msq[,'sleepy']) &
       complete.cases(msq[,'strong']) & complete.cases(msq[,'sociable']) & complete.cases(msq[,'happy'])
new_msq <- msq[sel, ]

#### Linear Model 1

Using multiple linear regression, fit a general linear model to predict `happy` using all affects that start with i : `idle` + `inspired` + `intense` + `interested` + `irritable`

In [ ]:
i_mod <- lm(happy ~ idle + inspired + intense + interested + irritable, data=new_msq)
summary(i_mod)

#### Linear Model 2

In this model, we will combine all these predictors and use: `idle`, `inspired`, `intense`, `interested`, `irritable`, `satisfied`, `scared`, `sleepy`, `strong` and `sociable`.

In [ ]:
is_mod <- lm(happy ~ idle + inspired + intense + interested + irritable +
             satisfied + scared + sleepy + strong + sociable, data = new_msq)
summary(is_mod)

#### Difference in MSE

This is another measure we will take a look at: did the new model reduce error/noise?

In [ ]:
mse1 <- mean(i_mod$residuals^2)
mse2 <- mean(is_mod$residuals^2)

mse1-mse2

### Cross Validation

We want to first partition the data into a training set and a test set. We will do a 75-25 split.

In [ ]:
require(caTools)
set.seed(101)

sample <- sample.split(new_msq[,1], SplitRatio = .75)
train <- subset(new_msq, sample == TRUE)
test <- subset(new_msq, sample == FALSE)

In [ ]:
nrow(train)

In [ ]:
nrow(test)

Now we will run our linear model through cross validation to see if the statistics we got is representative when the data is shuffled. We will do a 20-fold CV.

Again, **ONLY USE TRAINING DATA HERE**.

In [ ]:
n.folds <- 20
folds <- cut(seq(1,nrow(train)),breaks=n.folds,labels=FALSE)

We will create some empty arrays to collect the models and the statistics.

In [ ]:
MSE.imod <- array(data=0, dim = n.folds)    # Space for MSE
MSE.ismod <- array(data=0, dim = n.folds)

DE.i.is <- array(data=0, dim = n.folds)     # Space for differences in MSE
R2.i.is <- array(data=0, dim = n.folds)     # Space for R2

Now we will use a for-loop to help us run cross validation.

In [ ]:
for(i in 1:n.folds){
    
    #Segement your data by fold using the which() function 
    validateIdx <- which(folds==i,arr.ind=TRUE)
    validateData <- train[validateIdx, ]
    trainData <- train[-validateIdx, ]

    # Fit the threes models
    i_lmod <- lm(happy ~ idle + inspired + intense + interested + irritable, 
                 data=trainData)
    is_lmod <- lm(happy ~ idle + inspired + intense + interested + irritable + 
                  satisfied + scared + sleepy + strong + sociable,
                  data = trainData)
    
    # Get predictions with validation data
    i_preds <- predict(i_lmod, validateData)
    is_preds <- predict(is_lmod, validateData)
    
    # Calculate the mean square errors.
    y <- validateData$happy
    MSE.imod[i] <- mean((y - i_preds)^2)
    MSE.ismod[i] <- mean((y - is_preds)^2)

    # Differences in error: (note - this can also be calculated outside the loop)
    DE.i.is[i] <- MSE.imod[i] - MSE.ismod[i]
    
    # Calculate R2 for the nested models. (this can also be done outside the loop)
    R2.i.is[i] <- 1 - (MSE.ismod[i]/MSE.imod[i])
}

We can then look at the averaged results from our cross validation, and compare it with our ANOVA. 

In [ ]:
paste0("The average MSE for the model 1 was ", round(mean(MSE.imod),3))
paste0("The average MSE for the model 2 was ", round(mean(MSE.ismod),3))
paste0("The average R-squared for comparison between model 1 and 2 was ",
       round(mean(R2.i.is),3))

We can plot and see what the average differences in MSE are, and where our original model stand.

In [ ]:
mean_i_is <- mean(DE.i.is)
se_i_is <- sd(DE.i.is)
c(mean_i_is-2*se_i_is, mean_i_is+2*se_i_is)
hist(DE.i.is)
abline(v=c(mean_i_is-2*se_i_is, mean_i_is+2*se_i_is), col="red")

abline(v=c(mse1-mse2), col="blue")

#### Testing

Now let's run our 2 models to predict our test data, and see how we do.

In [ ]:
i_preds_test <- predict(i_lmod, test)
is_preds_test <- predict(is_lmod, test)

In [ ]:
y <- test$happy
mse1_test <- mean((y - i_preds_test)^2)
mse2_test <- mean((y - is_preds_test)^2)

mse1_test - mse2_test

This is actually pretty close to what we had with our training set original models, as well as our cross validation results. We can plot this onto our histogram as well.

In [ ]:
mean_i_is <- mean(DE.i.is)
se_i_is <- sd(DE.i.is)
c(mean_i_is-2*se_i_is, mean_i_is+2*se_i_is)
hist(DE.i.is)
abline(v=c(mean_i_is-2*se_i_is, mean_i_is+2*se_i_is), col="red")

abline(v=c(mse1-mse2), col="blue")
abline(v=c(mse1_test-mse2_test), col="green")

#### Exercise

There is another statistic in the model we have computed and stored, the $R^2$ of the nested model. Repeat this whole process and see if we can get similar results for $R^2$ as the differences in MSE we have calculated here. 